# Image Classification CNN

In [56]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from sklearn import decomposition
from sklearn import manifold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from tqdm.notebook import tqdm, trange
import matplotlib.pyplot as plt
import numpy as np

import copy
import random
import time

In [57]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [58]:
ROOT = '../../data'

train_data = datasets.CIFAR10(root=ROOT,
                            train=True,
                            download=True)

train_data.data = torch.tensor(train_data.data)

channels = train_data.data.split(1, dim=-1)
channel_tensors = [channel.squeeze(-1) for channel in channels]

means = [z.float().mean() / 255 for z in channel_tensors]
stds = [z.float().std() / 255 for z in channel_tensors]


print(f'Calculated mean: {means}')
print(f'Calculated std: {stds}')

Files already downloaded and verified
Calculated mean: [tensor(0.4914), tensor(0.4822), tensor(0.4465)]
Calculated std: [tensor(0.2470), tensor(0.2435), tensor(0.2616)]


In [59]:
train_transforms = transforms.Compose([
                            # transforms.RandomRotation(5, fill=(0,)),
                            # transforms.RandomCrop(28, padding=2),
                            transforms.ToTensor(),
                            transforms.Normalize(mean=means, std=stds)
                                      ])

test_transforms = transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize(mean=means, std=stds)
                                     ])

In [60]:
train_data = datasets.CIFAR10(root=ROOT,
                            train=True,
                            download=True,
                            transform=train_transforms)

test_data = datasets.CIFAR10(root=ROOT,
                           train=False,
                           download=True,
                           transform=test_transforms)

Files already downloaded and verified
Files already downloaded and verified


In [61]:
VALID_RATIO = 0.9

n_train_examples = int(len(train_data) * VALID_RATIO)
n_valid_examples = len(train_data) - n_train_examples

train_data, valid_data = data.random_split(train_data,
                                           [n_train_examples, n_valid_examples])


In [62]:
valid_data = copy.deepcopy(valid_data)
valid_data.dataset.transform = test_transforms

In [63]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 45000
Number of validation examples: 5000
Number of testing examples: 10000


In [64]:
BATCH_SIZE = 64

train_iterator = data.DataLoader(train_data, batch_size=BATCH_SIZE,shuffle = True)
valid_iterator = data.DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle = True)
test_iterator = data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)



In [65]:
class Model(nn.Module):
    def __init__(self, output_classes=10):
        super().__init__()
        self.conv1 = nn.LazyConv2d(out_channels=128,kernel_size=11,padding=5)
        self.conv2 = nn.LazyConv2d(out_channels = 256, kernel_size=7, padding=3)
        self.conv3 = nn.LazyConv2d(out_channels = 512, kernel_size=3, padding = 1)
        
        self.MLP = nn.LazyLinear(out_features=1024)
        self.end = nn.LazyLinear(out_features=output_classes)


    def init_weights(self):
        nn.init.xavier_uniform_(self.conv1.weight)
        nn.init.xavier_uniform_(self.conv2.weight)
        nn.init.xavier_uniform_(self.conv3.weight)
        nn.init.xavier_uniform_(self.MLP.weight)
        nn.init.xavier_uniform_(self.end.weight)

    def forward(self, X):
        X = nn.ReLU()(self.conv1(X))
        X = nn.MaxPool2d(kernel_size=3,padding=1)(X)
        X = nn.ReLU()(self.conv2(X))
        X = nn.MaxPool2d(kernel_size=3, padding=1)(X)
        X = nn.ReLU()(self.conv3(X))
        X = nn.MaxPool2d(kernel_size=3, padding=1)(X)
        X = nn.Flatten()(X)
        X = nn.ReLU()(self.MLP(X))
        X = self.end(X)
        return X


model = Model()
model(torch.Tensor(1,3,32,32))

def init_cnn(module: nn.Module):
    if type(module) == nn.Linear or type(module) == nn.Conv2d:
        nn.init.xavier_uniform_(module.weight)
        

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f'The model has {count_parameters(model):,} trainable parameters')


The model has 4,941,066 trainable parameters


In [66]:
optimizer = optim.Adam(model.parameters())
loss = nn.CrossEntropyLoss()
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
print(device)

mps


In [67]:
model = model.to(device)
loss = loss.to(device)

In [68]:
def acc(y_hat: torch.Tensor, y: torch.Tensor):
    preds = y_hat.argmax(1, keepdim=True)
    hits = preds.eq(y.view_as(preds)).sum()
    return hits.float() / y.shape[0]

In [69]:
def train(model, it, optim, loss, device):
    epoch_loss = 0
    epoch_acc = 0
    model.train() # set training mode

    for (data, labels) in tqdm(it, desc="Training", leave=False):
        data = data.to(device)
        labels = labels.to(device)
        
        optim.zero_grad()
        y_hat = model(data)
        batch_loss = loss(y_hat, labels)
        batch_acc = acc(y_hat, labels)

        batch_loss.backward()
        optim.step()



        epoch_loss += batch_loss.item()
        epoch_acc += batch_acc.item()

    return epoch_loss / len(it), epoch_acc / len(it) # avg loss/acc

def eval(model, it, loss, device):
    epoch_loss = 0
    epoch_acc = 0
    model.eval() # set training mode

    with torch.no_grad():
        for (data, labels) in tqdm(it, desc="Evaluating", leave=False):
            data = data.to(device)
            labels = labels.to(device)

            y_hat = model(data)
            batch_loss = loss(y_hat, labels)
            batch_acc = acc(y_hat, labels)

            epoch_loss += batch_loss.item()
            epoch_acc += batch_acc.item()

    return epoch_loss / len(it), epoch_acc / len(it) # avg loss/acc


In [70]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [71]:
EPOCHS = 69

for epoch in trange(EPOCHS, desc="Epochs"):
     start_time = time.monotonic()

     train_loss, train_acc = train(model, train_iterator, optimizer, loss, device)
     valid_loss, valid_acc = eval(model, valid_iterator, loss, device)
     end_time = time.monotonic()

     epoch_mins, epoch_secs = epoch_time(start_time, end_time)

     print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
     print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
     print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epochs:   0%|          | 0/69 [00:00<?, ?it/s]

Training:   0%|          | 0/704 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/79 [00:00<?, ?it/s]

Epoch: 01 | Epoch Time: 2m 9s
	Train Loss: 1.619 | Train Acc: 40.68%
	 Val. Loss: 1.346 |  Val. Acc: 52.27%


Training:   0%|          | 0/704 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/79 [00:00<?, ?it/s]

Epoch: 02 | Epoch Time: 2m 3s
	Train Loss: 1.257 | Train Acc: 54.87%
	 Val. Loss: 1.246 |  Val. Acc: 56.47%


Training:   0%|          | 0/704 [00:00<?, ?it/s]

KeyboardInterrupt: 

: 